In [1]:
! pip install librosa
! pip install jiwer
! pip install gradio
! pip  install datasets
! pip install evaluate
! pip install ipywidgets
! pip install huggingface_hub


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from datasets import load_dataset
import json

# Load dataset metadata
datasets = {
    "AB": "ganga4364/benchmark-stt-AB",
    "CS": "ganga4364/benchmark-stt-CS",
    "HS": "ganga4364/benchmark-stt-HS",
    "MV": "ganga4364/benchmark-stt-MV",
    "NS": "ganga4364/benchmark-stt-NS",
    "NW": "ganga4364/benchmark-stt-NW",
    "PC": "ganga4364/benchmark-stt-PC",
    "TT": "ganga4364/benchmark-stt-TT"
}

# Download and save datasets
for name, url in datasets.items():
    print(f"Downloading {name} dataset...")
    dataset = load_dataset(url)
    dataset.save_to_disk(f"./downloaded_datasets/{name}")


Saving the dataset (0/1 shards):   0%|          | 0/1680 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1607 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1707 [00:00<?, ? examples/s]

...

Saving the dataset (0/1 shards):   0%|          | 0/1229 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1995 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1888 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/348 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1831 [00:00<?, ? examples/s]

In [5]:
import pandas as pd
from datasets import load_from_disk, concatenate_datasets

# Assuming 'datasets' is your dictionary of dataset names and URLs
dataset_file_names = {}
all_file_names = []  # Initialize a single list to store all file names

for dataset_name, url in datasets.items():
    print(f"Processing {dataset_name} dataset...")
    
    # Load the dataset
    dataset = load_from_disk(f"./downloaded_datasets/{dataset_name}")
    
    # Initialize a list to store file names from all splits
    dataset_file_names[dataset_name] = []
    
    # Iterate through each split in the dataset
    for split_name in dataset.keys():
        # Extract file_name column from the current split
        file_names = dataset[split_name].to_pandas()['file_name'].tolist()
        dataset_file_names[dataset_name].extend(file_names)
        all_file_names.extend(file_names)  # Append to the single list of all file names

# Print the number of file names extracted for each dataset
for dataset_name, file_names in dataset_file_names.items():
    print(f"{dataset_name}: {len(file_names)} file names extracted")

print(f"Total number of file names across all datasets: {len(all_file_names)}")

# Optionally, you can save all file names to a single CSV file
all_file_names_df = pd.DataFrame({'file_name': all_file_names})
all_file_names_df.to_csv("benchmark_v3.csv", index=False)

Processing AB dataset...
Processing CS dataset...
Processing HS dataset...
Processing MV dataset...
Processing NS dataset...


Processing NW dataset...
Processing PC dataset...
Processing TT dataset...
AB: 7287 file names extracted
CS: 8350 file names extracted
HS: 9934 file names extracted
MV: 3267 file names extracted
NS: 8520 file names extracted
NW: 9863 file names extracted
PC: 11448 file names extracted
TT: 6062 file names extracted
Total number of file names across all datasets: 64731
